In [1]:
import pandas as pd
all_jobs = pd.read_csv("all_jobs.csv")

C:\Users\Admin\AppData\Local\Temp\ipykernel_21444\1021965397.py:2: DtypeWarning: Columns (1,6,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  all_jobs = pd.read_csv("all_jobs.csv")


In [ ]:
df = all_jobs.copy()
df.shape, df.columns

In [3]:
df.isna().mean().sort_values(ascending=False)

responsibilities    0.445337
qualifications      0.445337
country             0.445337
skills_raw          0.001042
company             0.000037
job_key             0.000034
location            0.000007
source              0.000000
job_title           0.000000
job_description     0.000000
dtype: float64

1-clean all jobs dataset

In [6]:
import pandas as pd
import re
from pathlib import Path


input_path  = "all_jobs.csv"               # merged big file (6GB)
output_path = "all_jobs_clean_full.csv"    # final cleaned file

chunksize = 50_000

# 🔐 for GLOBAL duplicate removal
seen_job_keys = set()       # for drop_duplicates(subset=["job_key"])
seen_composite_keys = set() # for drop_duplicates subset combo


# 4️⃣ your cleaning function, vectorized-style
def clean_text_series(s: pd.Series):
    return (
        s.astype(str)
         .str.replace("\n", " ", regex=False)
         .str.replace("\r", " ", regex=False)
         .str.replace(r"\s+", " ", regex=True)
         .str.strip()
    )

# 3️⃣ same logic: empty row = no title, no desc, no skills
def drop_fully_empty_rows(df: pd.DataFrame) -> pd.DataFrame:
    def is_empty(x):
        return (pd.isna(x)) or (str(x).strip() == "")

    mask_keep = ~(
        df["job_title"].apply(is_empty) &
        df["job_description"].apply(is_empty) &
        df["skills_raw"].apply(is_empty)
    )
    return df[mask_keep].copy()


def make_composite_key(row):
    # like your subset=["job_title","company","location","job_description"]
    parts = [
        str(row.get("job_title", "")).lower(),
        str(row.get("company", "")).lower(),
        str(row.get("location", "")).lower(),
        str(row.get("job_description", "")).lower()[:200],  # truncated desc to reduce memory
    ]
    return "||".join(parts)


def process_chunk(chunk: pd.DataFrame, first: bool):
    global seen_job_keys, seen_composite_keys

    # drop useless very sparse columns if they still exist
    for col in ["responsibilities", "qualifications","country"]:
        if col in chunk.columns:
            chunk = chunk.drop(columns=[col])

    # basic missing handling for text columns
    for col in ["job_title", "job_description", "skills_raw"]:
        if col in chunk.columns:
            chunk[col] = chunk[col].fillna("")

    # 👉 step 3: drop rows with empty title+desc+skills
    chunk = drop_fully_empty_rows(chunk)

    # 👉 step 4: clean text (job_title, job_description, skills_raw, location, company, country)
    cols_to_clean = ["job_title", "job_description", "skills_raw",
                     "location", "company", "country"]
    for col in cols_to_clean:
        if col in chunk.columns:
            chunk[col] = clean_text_series(chunk[col])

    # 👉 step 5: filter short title / desc
    if "job_title" in chunk.columns:
        chunk = chunk[chunk["job_title"].str.len() >= 5]
    if "job_description" in chunk.columns:
        chunk = chunk[chunk["job_description"].str.len() >= 30]

    # 👉 step 6.1: global dedup by job_key (if exists)
    if "job_key" in chunk.columns:
        mask_new_key = ~chunk["job_key"].isin(seen_job_keys)
        chunk = chunk[mask_new_key].copy()
        seen_job_keys.update(chunk["job_key"].tolist())

    # 👉 step 6.2: global dedup by combo (title+company+location+desc)
    #     (does what your drop_duplicates(subset=[...]) did, but globally)
    chunk["__dedup_key__"] = chunk.apply(make_composite_key, axis=1)
    mask_new_combo = ~chunk["__dedup_key__"].isin(seen_composite_keys)
    chunk = chunk[mask_new_combo].copy()
    seen_composite_keys.update(chunk["__dedup_key__"].tolist())
    chunk = chunk.drop(columns=["__dedup_key__"])

    return chunk


def run_full_clean():
    first = True

    for chunk in pd.read_csv(input_path, chunksize=chunksize):
        print("Processing new chunk...")
        cleaned_chunk = process_chunk(chunk, first)

        cleaned_chunk.to_csv(
            output_path,
            mode="w" if first else "a",
            header=first,
            index=False
        )
        first = False

    print("✅ Finished: full cleaned dataset written to:", output_path)


if __name__ == "__main__":
    run_full_clean()


Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new chunk...
Processing new c

2- clean skill migration public dataset 

In [7]:
import pandas as pd

# 1. Load raw file
skills_raw = pd.read_csv("skill_migration_public.csv")
print(skills_raw.shape)
print(skills_raw.head())

(17617, 12)
  country_code country_name   wb_income   wb_region  skill_group_id  \
0           af  Afghanistan  Low income  South Asia            2549   
1           af  Afghanistan  Low income  South Asia            2608   
2           af  Afghanistan  Low income  South Asia            3806   
3           af  Afghanistan  Low income  South Asia           50321   
4           af  Afghanistan  Low income  South Asia            1606   

          skill_group_category        skill_group_name  net_per_10K_2015  \
0                  Tech Skills  Information Management           -791.59   
1              Business Skills  Operational Efficiency          -1610.25   
2  Specialized Industry Skills       National Security          -1731.45   
3                  Tech Skills        Software Testing           -957.50   
4  Specialized Industry Skills                    Navy          -1510.71   

   net_per_10K_2016  net_per_10K_2017  net_per_10K_2018  net_per_10K_2019  
0           -705.88         

In [8]:
# 2. Keep only the useful columns
skills = skills_raw[[
    "skill_group_name",
    "skill_group_category"
]].copy()

# 3. Drop rows where skill name is missing
skills = skills[skills["skill_group_name"].notna()].copy()

# 4. Normalize text (lowercase, strip)
for col in ["skill_group_name", "skill_group_category"]:
    if col in skills.columns:
        skills[col] = (
            skills[col]
            .astype(str)
            .str.strip()
            .str.lower()
        )

# 5. Drop exact duplicates on (skill_group_name, skill_group_category)
skills = skills.drop_duplicates(subset=["skill_group_name", "skill_group_category"])

# 6. If one skill appears with multiple categories, we keep the first one
skills = skills.drop_duplicates(subset=["skill_group_name"], keep="first")

print(skills.shape)
print(skills.head())


(249, 2)
         skill_group_name         skill_group_category
0  information management                  tech skills
1  operational efficiency              business skills
2       national security  specialized industry skills
3        software testing                  tech skills
4                    navy  specialized industry skills


In [9]:
skills.to_csv("skill_migration_clean.csv", index=False)
print("✅ Saved cleaned skills dictionary to skill_migration_clean.csv")

✅ Saved cleaned skills dictionary to skill_migration_clean.csv


3- Map skills onto all_jobs_clean_full.csv with chunks

Cell 1: Load skills dictionary

In [1]:
import pandas as pd
import re

skills_meta = pd.read_csv("skill_migration_clean.csv")

skill_dict = (
    skills_meta[["skill_group_name", "skill_group_category"]]
    .drop_duplicates()
    .set_index("skill_group_name")["skill_group_category"]
    .to_dict()
)

print(f"Loaded {len(skill_dict)} skills.")


Loaded 249 skills.


Cell 2: Define helper functions

In [2]:
def parse_skills(raw):
    if pd.isna(raw):
        return []
    text = str(raw).strip().lower()
    if not text:
        return []
    parts = re.split(r",|/|;|\||\+", text)
    return [p.strip() for p in parts if p.strip()]

def map_skill_categories(skill_list):
    cats = set()
    for s in skill_list:
        cat = skill_dict.get(s)
        if cat is None:
            cat = "other"
        cats.add(cat)
    if not cats:
        return ""
    return ",".join(sorted(cats))


Cell 3: Process all_jobs_clean_full.csv with chunks

In [3]:
input_path  = "all_jobs_clean_full.csv"
output_path = "all_jobs_mapped.csv"
chunksize   = 50_000

first = True

for chunk in pd.read_csv(input_path, chunksize=chunksize):
    print(f"Processing {len(chunk)} rows...")

    if "skills_raw" not in chunk.columns:
        chunk["skills_raw"] = ""

    chunk["skill_list"] = chunk["skills_raw"].apply(parse_skills)
    chunk["skill_categories"] = chunk["skill_list"].apply(map_skill_categories)

    chunk.to_csv(
        output_path,
        mode="w" if first else "a",
        header=first,
        index=False
    )
    first = False

print("✅ Done ! Final dataset:", output_path)


Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...
Processing 50000 rows...


In [4]:
import os
print(os.path.getsize("all_jobs_mapped.csv") / (1024*1024), "MB")


6904.832558631897 MB


In [2]:
import pandas as pd
df = pd.read_csv("all_jobs_mapped.csv")
df.shape


C:\Users\dell\AppData\Local\Temp\ipykernel_23180\1045503748.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("all_jobs_mapped.csv")


(2884556, 9)

In [9]:
df.columns

Index(['source', 'job_key', 'job_title', 'job_description', 'skills_raw',
       'location', 'company', 'skill_list', 'skill_categories'],
      dtype='object')

TRAINING STEPS TO FIND MODELS 

In [2]:
import pandas as pd

path = "all_jobs_mapped.csv"

# Load a manageable sample for experiments (you can increase nrows later)
df = pd.read_csv(path)

df.shape, df.columns


C:\Users\dell\AppData\Local\Temp\ipykernel_4476\3013172477.py:6: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


((2884556, 9),
 Index(['source', 'job_key', 'job_title', 'job_description', 'skills_raw',
        'location', 'company', 'skill_list', 'skill_categories'],
       dtype='object'))

--Ensure skill_list is really a lis

In [3]:
import ast

def to_skill_list(x):
    if isinstance(x, list):
        return x
    if pd.isna(x):
        return []
    try:
        val = ast.literal_eval(x)
        if isinstance(val, list):
            return [str(s).strip().lower() for s in val]
    except:
        pass
    return [s.strip().lower() for s in str(x).split(",") if s.strip()]

df["skill_list"] = df["skill_list"].apply(to_skill_list)
df["skill_categories"] = df["skill_categories"].fillna("").astype(str)


Association Rules: 3 Models 

Model A1: skills only

In [4]:
transactions_skills = df["skill_list"].tolist()

Model A2: categories only

In [5]:
def cat_list(cats_str):
    if not cats_str:
        return []
    return [c.strip().lower() for c in cats_str.split(",") if c.strip()]

df["cat_list"] = df["skill_categories"].apply(cat_list)
transactions_cats = df["cat_list"].tolist()



Model A3: skills + categories

In [6]:
def combined_list(row):
    return row["skill_list"] + row["cat_list"]

df["combined_list"] = df.apply(combined_list, axis=1)
transactions_combined = df["combined_list"].tolist()



One-hot encode and run Apriori / FP-Growth

A1

In [7]:
# Replace apriori with a memory-friendly FP-Growth pipeline
from collections import Counter
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

# --- Parameters you can tune ---
min_support = 0.01        # same as your original apriori support
min_confidence = 0.4      # same as before
min_occurrences = 10      # absolute min occurrences for a skill (safety net)
# -------------------------------

n_transactions = len(transactions_skills)
# derive an absolute min occurrence threshold from min_support (safer)
min_occ_from_support = max(1, int(min_support * n_transactions))
min_keep = max(min_occurrences, min_occ_from_support)

# 1) Count skill frequencies and filter rare skills
skill_counts = Counter(skill for tx in transactions_skills for skill in tx)
valid_skills = {skill for skill, cnt in skill_counts.items() if cnt >= min_keep}
filtered_transactions = [[s for s in tx if s in valid_skills] for tx in transactions_skills]

if len(valid_skills) == 0:
    raise ValueError("No skills left after filtering. Lower min_keep or min_support.")

# 2) Encode transactions as a sparse matrix to save memory
te = TransactionEncoder()
# fit on filtered transactions (much smaller vocabulary)
te_ary = te.fit(filtered_transactions).transform(filtered_transactions, sparse=True)

# convert to a pandas SparseDataFrame (saves memory compared to dense)
skills_df = pd.DataFrame.sparse.from_spmatrix(te_ary, columns=te.columns_)

# 3) Run FP-Growth (more efficient than Apriori on large data)
freq_itemsets_A1 = fpgrowth(skills_df, min_support=min_support, use_colnames=True)

# 4) Derive association rules
rules_A1 = association_rules(freq_itemsets_A1, metric="confidence", min_threshold=min_confidence)

# Optional: quick diagnostics
print(f"Transactions: {n_transactions:,}")
print(f"Kept skills: {len(valid_skills):,}")
print(f"Frequent itemsets: {len(freq_itemsets_A1):,}")
print(f"Rules: {len(rules_A1):,}")


C:\Users\dell\AppData\Local\Temp\ipykernel_4476\2700307426.py:32: FutureWarning: Allowing arbitrary scalar fill_value in SparseDtype is deprecated. In a future version, the fill_value must be a valid value for the SparseDtype.subtype.
  skills_df = pd.DataFrame.sparse.from_spmatrix(te_ary, columns=te.columns_)


Transactions: 2,884,556
Kept skills: 79
Frequent itemsets: 155
Rules: 73


A2

In [8]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

te = TransactionEncoder()
te_ary = te.fit(transactions_cats).transform(transactions_cats)
cats_df = pd.DataFrame(te_ary, columns=te.columns_)

freq_itemsets_A2 = apriori(cats_df, min_support=0.01, use_colnames=True)
rules_A2 = association_rules(freq_itemsets_A2, metric="confidence", min_threshold=0.4)



In [9]:
print(f"Rules: {len(rules_A2):,}")

Rules: 17


A3

In [10]:
from collections import Counter
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

# Parameters
min_support = 0.01
min_confidence = 0.4
min_occurrences = 10  # remove very rare categories

n_transactions = len(transactions_combined)
min_occ_from_support = max(1, int(min_support * n_transactions))
min_keep = max(min_occurrences, min_occ_from_support)

# 1) Filter rare categories
cat_counts = Counter(cat for tx in transactions_combined for cat in tx)
valid_cats = {cat for cat, cnt in cat_counts.items() if cnt >= min_keep}
filtered_transactions = [[c for c in tx if c in valid_cats] for tx in transactions_combined]

# 2) Encode as sparse matrix
te = TransactionEncoder()
te_ary = te.fit(filtered_transactions).transform(filtered_transactions, sparse=True)
cats_df = pd.DataFrame.sparse.from_spmatrix(te_ary, columns=te.columns_)

# 3) Use FP-Growth instead of Apriori
freq_itemsets_A3 = fpgrowth(cats_df, min_support=min_support, use_colnames=True)

# 4) Generate rules
rules_A3 = association_rules(freq_itemsets_A3, metric="confidence", min_threshold=min_confidence)

# Diagnostics
print(f"Transactions: {n_transactions:,}")
print(f"Kept categories: {len(valid_cats):,}")
print(f"Frequent itemsets: {len(freq_itemsets_A3):,}")
print(f"Rules: {len(rules_A3):,}")


C:\Users\dell\AppData\Local\Temp\ipykernel_4476\3888980484.py:23: FutureWarning: Allowing arbitrary scalar fill_value in SparseDtype is deprecated. In a future version, the fill_value must be a valid value for the SparseDtype.subtype.
  cats_df = pd.DataFrame.sparse.from_spmatrix(te_ary, columns=te.columns_)


Transactions: 2,884,556
Kept categories: 84
Frequent itemsets: 693
Rules: 1,801


In [11]:
def summarize_rules(rules, name):
    print("=== ", name, " ===")
    print("n_rules:", len(rules))
    print("support range:", rules["support"].min(), "→", rules["support"].max())
    print("confidence range:", rules["confidence"].min(), "→", rules["confidence"].max())
    print("lift range:", rules["lift"].min(), "→", rules["lift"].max())
    print()

summarize_rules(rules_A1, "A1: skill-level")
summarize_rules(rules_A2, "A2: category-level")
summarize_rules(rules_A3, "A3: combined")



===  A1: skill-level  ===
n_rules: 73
support range: 0.010083007575515954 → 0.0476239670854024
confidence range: 0.4012219636893173 → 0.8577052868391452
lift range: 3.2580860570026773 → 56.55732135912904

===  A2: category-level  ===
n_rules: 17
support range: 0.01090774455410122 → 0.20140083950528262
confidence range: 0.5202852510010257 → 1.0
lift range: 1.0008216365247928 → 3.1797653362987273

===  A3: combined  ===
n_rules: 1801
support range: 0.010000499210277075 → 0.20149374808462722
confidence range: 0.4012219636893173 → 1.0
lift range: 1.0006534533036906 → 56.55732135912904



2️⃣ Clustering: 3 Models (KMeans, DBSCAN, Agglomerative)

Step 2.0 – Load your data and import libraries


In [2]:
import pandas as pd
import ast
from collections import Counter
from mlxtend.preprocessing import TransactionEncoder
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_score
import numpy as np
from sklearn.feature_extraction import FeatureHasher
import joblib

# Load your cleaned dataset
df = pd.read_csv("all_jobs_mapped.csv")

# Ensure skill_list is properly formatted as list
def to_skill_list(x):
    if isinstance(x, list):
        return x
    if pd.isna(x):
        return []
    try:
        val = ast.literal_eval(x)
        if isinstance(val, list):
            return [str(s).strip().lower() for s in val]
    except:
        pass
    return [s.strip().lower() for s in str(x).split(",") if s.strip()]

df["skill_list"] = df["skill_list"].apply(to_skill_list)

C:\Users\dell\AppData\Local\Temp\ipykernel_26700\1661242774.py:12: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("all_jobs_mapped.csv")


Step 2.1 – Build job × skill matrix (MiniBatch Approach)

In [3]:
def build_minibatch_feature_matrix(df, chunk_size=50000, min_freq=50, n_features=1000):
    """Build feature matrix using MiniBatch approach for entire dataset"""
    print("🔍 Building global skill vocabulary from ALL data...")
    
    # Step 1: Build vocabulary from entire dataset in chunks
    all_skills = Counter()
    total_chunks = (len(df) + chunk_size - 1) // chunk_size
    
    for i in range(0, len(df), chunk_size):
        chunk = df.iloc[i:i+chunk_size]
        for skills in chunk["skill_list"]:
            all_skills.update(skills)
        if i % 100000 == 0:
            print(f"  Processed {i:,} jobs...")
    
    # Keep most frequent skills
    keep_skills = {k for k, v in all_skills.items() if v >= min_freq}
    skill_vocab = sorted(list(keep_skills))
    
    print(f"✅ Global vocabulary: {len(skill_vocab)} skills (appearing ≥ {min_freq} times)")
    
    # Step 2: Use FeatureHasher for memory-efficient transformation
    print("🔄 Transforming data with FeatureHasher...")
    
    def skills_to_dict(skills_list):
        return {skill: 1 for skill in skills_list if skill in keep_skills}
    
    # Process in chunks to avoid memory issues
    hasher = FeatureHasher(n_features=n_features, input_type='dict', alternate_sign=False)
    
    # Process in chunks and store results
    X_chunks = []
    
    for i in range(0, len(df), chunk_size):
        chunk = df.iloc[i:i+chunk_size]
        chunk_dicts = chunk["skill_list"].apply(skills_to_dict).tolist()
        X_chunk = hasher.transform(chunk_dicts)
        X_chunks.append(X_chunk)
    
    # Combine all chunks
    from scipy.sparse import vstack
    X_sparse = vstack(X_chunks)
    
    print(f"✅ Feature matrix built: {X_sparse.shape}")
    print(f"💾 Memory usage: {X_sparse.data.nbytes / (1024**2):.2f} MB")
    
    return X_sparse, skill_vocab, hasher

# Build the feature matrix for entire dataset
X_sparse, skill_vocab, hasher = build_minibatch_feature_matrix(df, chunk_size=50000, min_freq=50, n_features=1000)

# Create a smaller dense sample for models that require dense matrices
sample_size = min(20000, X_sparse.shape[0])
X_dense_sample = X_sparse[:sample_size].toarray()

print(f"Created dense sample: {X_dense_sample.shape} for models requiring dense matrices")

🔍 Building global skill vocabulary from ALL data...
  Processed 0 jobs...
  Processed 100,000 jobs...
  Processed 200,000 jobs...
  Processed 300,000 jobs...
  Processed 400,000 jobs...
  Processed 500,000 jobs...
  Processed 600,000 jobs...
  Processed 700,000 jobs...
  Processed 800,000 jobs...
  Processed 900,000 jobs...
  Processed 1,000,000 jobs...
  Processed 1,100,000 jobs...
  Processed 1,200,000 jobs...
  Processed 1,300,000 jobs...
  Processed 1,400,000 jobs...
  Processed 1,500,000 jobs...
  Processed 1,600,000 jobs...
  Processed 1,700,000 jobs...
  Processed 1,800,000 jobs...
  Processed 1,900,000 jobs...
  Processed 2,000,000 jobs...
  Processed 2,100,000 jobs...
  Processed 2,200,000 jobs...
  Processed 2,300,000 jobs...
  Processed 2,400,000 jobs...
  Processed 2,500,000 jobs...
  Processed 2,600,000 jobs...
  Processed 2,700,000 jobs...
  Processed 2,800,000 jobs...
✅ Global vocabulary: 40234 skills (appearing ≥ 50 times)
🔄 Transforming data with FeatureHasher...
✅ Fea

Step 2.2 – Model C1: K-Means

In [4]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

k_values = [5, 8, 10]  # Reduced for faster computation
results_kmeans = {}

print("KMeans Clustering Results:")
print("=" * 40)

for k in k_values:
    print(f"Running KMeans with k={k}...")
    km = KMeans(n_clusters=k, random_state=42, n_init=3)
    
    # KMeans works with sparse matrices, so use larger sample
    kmeans_sample_size = min(50000, X_sparse.shape[0])
    X_kmeans = X_sparse[:kmeans_sample_size]
    
    labels = km.fit_predict(X_kmeans)
    results_kmeans[k] = labels
    
    # Evaluate with silhouette score
    score = silhouette_score(X_kmeans, labels)
    results_kmeans[f"silhouette_{k}"] = score
    print(f"KMeans k={k}: silhouette={score:.3f}")

# Find best K
best_k_kmeans = max(k_values, key=lambda k: results_kmeans[f"silhouette_{k}"])
print(f"\nBest KMeans: k={best_k_kmeans} (silhouette={results_kmeans[f'silhouette_{best_k_kmeans}']:.3f})")

KMeans Clustering Results:
Running KMeans with k=5...
KMeans k=5: silhouette=0.010
Running KMeans with k=8...
KMeans k=8: silhouette=0.007
Running KMeans with k=10...
KMeans k=10: silhouette=0.005

Best KMeans: k=5 (silhouette=0.010)


Step 2.3 – Model C2: DBSCAN

In [5]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score

# DBSCAN works with sparse matrices
dbscan_sample_size = min(20000, X_sparse.shape[0])
X_dbscan = X_sparse[:dbscan_sample_size]

print(f"DBSCAN running on {X_dbscan.shape[0]} samples")
print("DBSCAN Clustering Results:")
print("=" * 40)

eps_values = [0.5, 0.7]
min_samples_values = [10, 15]

results_dbscan = {}

for eps in eps_values:
    for ms in min_samples_values:
        print(f"Testing DBSCAN eps={eps}, min_samples={ms}...")
        db = DBSCAN(eps=eps, min_samples=ms, metric='euclidean', n_jobs=-1)
        labels = db.fit_predict(X_dbscan)
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        n_noise = (labels == -1).sum()
        noise_percentage = (n_noise / len(labels)) * 100
        
        # Store results
        key = f"eps{eps}_min{ms}"
        results_dbscan[key] = {
            'labels': labels,
            'n_clusters': n_clusters,
            'n_noise': n_noise,
            'noise_percentage': noise_percentage
        }
        
        print(f"DBSCAN eps={eps}, min_samples={ms}:")
        print(f"  Clusters: {n_clusters}, Noise: {n_noise} ({noise_percentage:.1f}%)")
        
        # silhouette only if >1 cluster and not all noise
        if n_clusters > 1 and n_clusters < len(labels):
            mask = labels != -1
            if len(set(labels[mask])) > 1 and sum(mask) > 1:
                score = silhouette_score(X_dbscan[mask], labels[mask])
                results_dbscan[key]['silhouette'] = score
                print(f"  Silhouette: {score:.3f}")
        print()

DBSCAN running on 20000 samples
DBSCAN Clustering Results:
Testing DBSCAN eps=0.5, min_samples=10...
DBSCAN eps=0.5, min_samples=10:
  Clusters: 4, Noise: 19800 (99.0%)
  Silhouette: 1.000

Testing DBSCAN eps=0.5, min_samples=15...
DBSCAN eps=0.5, min_samples=15:
  Clusters: 2, Noise: 19824 (99.1%)
  Silhouette: 1.000

Testing DBSCAN eps=0.7, min_samples=10...
DBSCAN eps=0.7, min_samples=10:
  Clusters: 4, Noise: 19800 (99.0%)
  Silhouette: 1.000

Testing DBSCAN eps=0.7, min_samples=15...
DBSCAN eps=0.7, min_samples=15:
  Clusters: 2, Noise: 19824 (99.1%)
  Silhouette: 1.000



Step 2.4 – Model C3: Agglomerative clustering

In [6]:
from sklearn.cluster import AgglomerativeClustering

print("Agglomerative Clustering Results:")
print("=" * 40)

k_values = [5, 8, 10]
results_agglo = {}

for k in k_values:
    print(f"Running Agglomerative with k={k}...")
    ac = AgglomerativeClustering(n_clusters=k)
    
    # Agglomerative requires dense matrix, so use our dense sample
    labels = ac.fit_predict(X_dense_sample)
    results_agglo[k] = labels
    
    score = silhouette_score(X_dense_sample, labels)
    results_agglo[f"silhouette_{k}"] = score
    print(f"Agglomerative k={k}: silhouette={score:.3f}")

# Find best K for Agglomerative
best_k_agglo = max(k_values, key=lambda k: results_agglo[f"silhouette_{k}"])
print(f"\nBest Agglomerative: k={best_k_agglo} (silhouette={results_agglo[f'silhouette_{best_k_agglo}']:.3f})")

Agglomerative Clustering Results:
Running Agglomerative with k=5...
Agglomerative k=5: silhouette=-0.047
Running Agglomerative with k=8...
Agglomerative k=8: silhouette=-0.044
Running Agglomerative with k=10...
Agglomerative k=10: silhouette=-0.053

Best Agglomerative: k=8 (silhouette=-0.044)


Step 2.5 – Compare clustering models


In [7]:
def compare_all_models(results_kmeans, results_dbscan, results_agglo, k_values):
    """Compare all clustering models and select the best one"""
    
    print("CLUSTERING MODEL COMPARISON")
    print("=" * 50)
    
    best_model = None
    best_score = -1
    best_config = None
    
    # Compare KMeans models
    print("KMeans Results:")
    for k in k_values:
        score = results_kmeans[f"silhouette_{k}"]
        print(f"  k={k}: silhouette={score:.3f}")
        if score > best_score:
            best_score = score
            best_model = "KMeans"
            best_config = k
    
    # Compare Agglomerative models  
    print("\nAgglomerative Results:")
    for k in k_values:
        score = results_agglo[f"silhouette_{k}"]
        print(f"  k={k}: silhouette={score:.3f}")
        if score > best_score:
            best_score = score
            best_model = "Agglomerative"
            best_config = k
    
    # Compare DBSCAN models
    print("\nDBSCAN Results:")
    for key, result in results_dbscan.items():
        if 'silhouette' in result:
            score = result['silhouette']
            print(f"  {key}: silhouette={score:.3f} (clusters: {result['n_clusters']}, noise: {result['noise_percentage']:.1f}%)")
            if score > best_score:
                best_score = score
                best_model = "DBSCAN"
                best_config = key
    
    print("=" * 50)
    print(f"🏆 BEST MODEL: {best_model}")
    print(f"📊 Best Configuration: {best_config}")
    print(f"🎯 Best Silhouette Score: {best_score:.3f}")
    
    return best_model, best_config, best_score

# Run comparison
best_model, best_config, best_score = compare_all_models(results_kmeans, results_dbscan, results_agglo, k_values)

CLUSTERING MODEL COMPARISON
KMeans Results:
  k=5: silhouette=0.010
  k=8: silhouette=0.007
  k=10: silhouette=0.005

Agglomerative Results:
  k=5: silhouette=-0.047
  k=8: silhouette=-0.044
  k=10: silhouette=-0.053

DBSCAN Results:
  eps0.5_min10: silhouette=1.000 (clusters: 4, noise: 99.0%)
  eps0.5_min15: silhouette=1.000 (clusters: 2, noise: 99.1%)
  eps0.7_min10: silhouette=1.000 (clusters: 4, noise: 99.0%)
  eps0.7_min15: silhouette=1.000 (clusters: 2, noise: 99.1%)
🏆 BEST MODEL: DBSCAN
📊 Best Configuration: eps0.5_min10
🎯 Best Silhouette Score: 1.000


Step 2.6 – Apply Best Model to Full Dataset

In [8]:
# STEP 2.6 – Apply DBSCAN to Full Dataset (OPTIMIZED VERSION)
print("🚀 STEP 2.6: Applying OPTIMIZED DBSCAN to FULL DATASET...")
print("=" * 50)

# Use more aggressive parameters for large dataset
eps = 0.7  # Reduced from 2.0 - tighter neighborhoods
min_samples = 50  # Increased from 10 - require more points to form clusters

print(f"Training DBSCAN on FULL dataset with OPTIMIZED parameters:")
print(f"  eps: {eps}, min_samples: {min_samples}")
print(f"  Dataset size: {X_sparse.shape[0]:,} jobs")
print(f"  This will take approximately 30-90 minutes...")

# Train DBSCAN with progress tracking
import time
start_time = time.time()

dbscan_full = DBSCAN(eps=eps, min_samples=min_samples, metric='euclidean', n_jobs=-1)
full_labels = dbscan_full.fit_predict(X_sparse)

end_time = time.time()
print(f"✅ DBSCAN completed in {(end_time - start_time)/60:.1f} minutes")

# Add clusters to the main dataframe
df_full = df.copy()
df_full['cluster'] = full_labels

# Calculate statistics
n_clusters = len(set(full_labels)) - (1 if -1 in full_labels else 0)
n_noise = (full_labels == -1).sum()
clustered_percentage = ((len(full_labels) - n_noise) / len(full_labels)) * 100

print(f"📊 Results:")
print(f"  Total jobs: {len(full_labels):,}")
print(f"  Number of clusters: {n_clusters}")
print(f"  Noise points: {n_noise:,} ({n_noise/len(full_labels)*100:.1f}%)")
print(f"  Clustered points: {len(full_labels)-n_noise:,} ({clustered_percentage:.1f}%)")

# Save results
df_full.to_csv("all_jobs_clustered_full.csv", index=False)
print(f"✅ Full dataset saved: all_jobs_clustered_full.csv")

joblib.dump({
    'best_model': 'DBSCAN',
    'parameters': {'eps': eps, 'min_samples': min_samples},
    'skill_vocab': skill_vocab,
    'feature_hasher': hasher,
    'trained_model': dbscan_full,
}, 'clustering_results.pkl')

print("✅ Model and metadata saved")

🚀 STEP 2.6: Applying OPTIMIZED DBSCAN to FULL DATASET...
Training DBSCAN on FULL dataset with OPTIMIZED parameters:
  eps: 0.7, min_samples: 50
  Dataset size: 2,884,556 jobs
  This will take approximately 30-90 minutes...


KeyboardInterrupt: 